## ひろしまQuest2020#stayhome【球種予測部門】

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [23]:
ALL_MERGE = 'intermediate/all_merge_7.f'
SUBMIT = 'submit/lgbm_7.csv'

In [24]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 250)

### 不要な列を削除

In [25]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [26]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [27]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 235)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,pos_pit,date_from_opening,elapsed_min,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,...,pit_ball_per_inning,pit_ball_per_batter,foreigner,batter_cnt,bat_game_cnt,batter_cnt_per_game,c_straight,c_curve,c_slider,c_shoot,c_fork,c_changeup,c_sinker,c_cutball,c_total,cat_game_cnt,cat_inning_cnt,cat_batter_cnt,cat_inning_per_game,cat_batter_per_game,cat_ball_per_game,cat_batter_per_inning,cat_ball_per_inning,cat_ball_per_batter,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0.0,1.0,1,1,0,0,0.000000,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,NaN,NaN,0,443.0,135.0,3.281481,0.419137,0.072469,0.23547,0.105491,0.052524,0.050307,0.000886,0.063715,18049.0,137.0,1114.0,4625.0,8.131387,33.759124,131.744526,4.151706,16.201975,3.902486,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0,0,0,0,0.0,1.0,1,1,0,0,0.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,NaN,NaN,0,443.0,135.0,3.281481,0.419137,0.072469,0.23547,0.105491,0.052524,0.050307,0.000886,0.063715,18049.0,137.0,1114.0,4625.0,8.131387,33.759124,131.744526,4.151706,16.201975,3.902486,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0,0,0,0,0.0,1.0,1,1,0,0,0.383333,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,0,443.0,135.0,3.281481,0.419137,0.072469,0.23547,0.105491,0.052524,0.050307,0.000886,0.063715,18049.0,137.0,1114.0,4625.0,8.131387,33.759124,131.744526,4.151706,16.201975,3.902486,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0,0,0,0,0.0,0.0,1,1,0,0,1.016667,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,NaN,NaN,0,443.0,135.0,3.281481,0.419137,0.072469,0.23547,0.105491,0.052524,0.050307,0.000886,0.063715,18049.0,137.0,1114.0,4625.0,8.131387,33.759124,131.744526,4.151706,16.201975,3.902486,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0,0,0,0,0.0,0.0,1,1,0,0,1.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,NaN,NaN,0,443.0,135.0,3.281481,0.419137,0.072469,0.23547,0.105491,0.052524,0.050307,0.000886,0.063715,18049.0,137.0,1114.0,4625.0,8.131387,33.759124,131.744526,4.151706,16.201975,3.902486,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681


### train
- 行数: 257117

In [7]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 235)

### test
- 行数: 521650

In [8]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 235)

In [9]:
del all_pitch
gc.collect()

60

In [10]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [11]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [12]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [13]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 6.9923570049658075, 
        'lambda_l2': 0.002378623984798833, 
        'num_leaves': 18, 
        'feature_fraction': 0.45199999999999996, 
        'bagging_fraction': 0.9799724836460725, 
        'bagging_freq': 4, 
        'min_child_samples': 20
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
print('Best CV score:', cv_results['multi_logloss-mean'][-1])

[100]	cv_agg's multi_logloss: 1.1503 + 0.00223976
[200]	cv_agg's multi_logloss: 1.12936 + 0.00212705
[300]	cv_agg's multi_logloss: 1.12208 + 0.00234981
[400]	cv_agg's multi_logloss: 1.11783 + 0.00238306
[500]	cv_agg's multi_logloss: 1.11483 + 0.00240385
[600]	cv_agg's multi_logloss: 1.11254 + 0.00242338
[700]	cv_agg's multi_logloss: 1.11091 + 0.00252705
[800]	cv_agg's multi_logloss: 1.1098 + 0.00254625
[900]	cv_agg's multi_logloss: 1.10924 + 0.00266447
[1000]	cv_agg's multi_logloss: 1.10884 + 0.00263152
[1100]	cv_agg's multi_logloss: 1.10864 + 0.00265597
[1200]	cv_agg's multi_logloss: 1.1086 + 0.00278125
[1300]	cv_agg's multi_logloss: 1.10876 + 0.00274595
Best num_boost_round: 1204
Best CV score: 1.108590082437519
CPU times: user 1h 43min 7s, sys: 5.72 s, total: 1h 43min 13s
Wall time: 12min 56s


In [14]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [15]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 32min 51s, sys: 1.19 s, total: 32min 52s
Wall time: 4min 6s


### Feature Importance

In [16]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
216,age_dif_p-b,1941
225,bmi_dif_p-c,1950
109,height_bat,1991
15,strike_cnt,2002
14,ball_cnt,2105
158,salary_year_bat,2157
133,bat_game_cnt_bat,2192
6,player_cnt_in_game,2207
218,salary_x_year_dif_p-b,2246
214,salary_dif_p-b,2250


In [17]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 7min 9s, sys: 1.02 s, total: 7min 10s
Wall time: 55.5 s


## 結果出力

In [18]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.255876,0.024690,0.131615,0.413773,0.003591,0.000015,0.000019,0.170422
1,1,0.281095,0.089675,0.118677,0.426788,0.004983,0.000019,0.000012,0.078750
2,2,0.272705,0.091344,0.137865,0.307513,0.093821,0.000081,0.000029,0.096641
3,3,0.254353,0.099258,0.170270,0.394037,0.016015,0.000035,0.000014,0.066018
4,4,0.177972,0.037185,0.264665,0.434665,0.009988,0.000033,0.000016,0.075477


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [31]:
submit.mean()

index    260824.500000
0             0.554315
1             0.063672
2             0.173569
3             0.047594
4             0.072866
5             0.039707
6             0.007942
7             0.040334
dtype: float64

In [32]:
submit.to_csv(SUBMIT, header=False, index=False)

### SIGNATE

In [34]:
'signate submit --competition-id=275 ~/work/signate/baseball/{} --note feature={}'.format(SUBMIT, column_cnt)

'signate submit --competition-id=275 ~/work/signate/baseball/submit/lgbm_7.csv --note feature=235'

### 評価結果
#### 7 (235)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.108590082437519
- Wall time: 12min 56s
- 評価結果  : 1.4946937

#### 6 (207)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1135
- Best CV score: 1.1086837935965734
- Wall time: 11min 54s
- 評価結果  : 1.3999194

#### 5-tune (参考)
- 'num_leaves' : 18,
- 'learning_rate' : 0.05,
- Best num_boost_round: 2395
- Best CV score: 1.1103399384119979
- Wall time: 18min 29s (n1-standard-8)

#### 5-tune (135)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- CV score: 1.1112402566616455
- 評価結果  : 1.3991780
- CV Wall time: 27min 48s (local) -> 18min 24s (n1-standard-4) -> 9min 3s (n1-standard-8)

#### 5 (135)
- 'num_leaves' : 31,
- 'learning_rate' : 0.1,
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806